# Capstone: Financial Research Copilot

## Step 0: Setup and Configuration
Importing necessary libraries (yfinance, pandas, google-generativeai) and configuring the Google Gemini client using the API key from `.env`.

In [35]:
import os
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from dotenv import load_dotenv
import google.generativeai as genai
import json

In [2]:
# 1. Load environment variables
load_dotenv()

# 2. Get API Key
api_key = os.getenv("GOOGLE_API_KEY")

if not api_key:
    print("Error: GOOGLE_API_KEY not found in .env file.")
else:
    print("API Key loaded successfully.")
    
    # 3. Configure the client
    try:
        genai.configure(api_key=api_key)
        print("Google GenAI Client configured successfully.")
    except Exception as e:
        print(f"Error configuring client: {e}")

API Key loaded successfully.
Google GenAI Client configured successfully.


## Step 1.1: Price History Tool
Defining `get_asset_price_history` to fetch OHLCV data for Stocks and ETFs using `yfinance`. This tool returns structured JSON data that the agent can analyze.


In [3]:
from typing import Dict, Any

def get_asset_price_history(symbol: str, asset_type: str = "equity", period: str = "1mo") -> Dict[str, Any]:
    """
    Fetch historical OHLCV price data for an asset.
    
    Args:
        symbol (str): Ticker symbol (e.g., 'AAPL', 'BTC-USD').
        asset_type (str): Type of asset ('equity', 'crypto', 'etf'). Defaults to 'equity'.
        period (str): Data period to download (e.g., '1mo', '3mo', '1y'). Defaults to '1mo'.
        
    Returns:
        dict: Structured data containing status and price history.
    """
    try:
        # Handle crypto symbols for yfinance (usually needs -USD suffix if not provided)
        if asset_type.lower() == 'crypto' and not symbol.endswith('-USD'):
            symbol = f"{symbol}-USD"
            
        ticker = yf.Ticker(symbol)
        
        # Fetch history
        hist = ticker.history(period=period)
        
        if hist.empty:
            return {
                "status": "error",
                "message": f"No data found for {symbol}. Check ticker or period."
            }
            
        # Format data for the agent (convert to list of dicts)
        # We limit to the last 60 records to keep context small if period is long
        data_records = []
        for date, row in hist.tail(60).iterrows():
            data_records.append({
                "date": date.strftime('%Y-%m-%d'),
                "close": round(row['Close'], 2),
                "volume": int(row['Volume'])
            })
            
        return {
            "status": "success",
            "symbol": symbol,
            "currency": ticker.info.get('currency', 'USD'),
            "data": data_records
        }

    except Exception as e:
        return {"status": "error", "message": str(e)}

In [ ]:
# --- Testing the tool  ---
print("Testing Tool 1.1 with AAPL...")
result = get_asset_price_history("AAPL", period="5d")
result = json.dumps(result, indent=4)
print(result)


Testing Tool 1.1 with AAPL...
{'status': 'success', 'symbol': 'AAPL', 'currency': 'USD', 'data': [{'date': '2025-11-20', 'close': np.float64(266.25), 'volume': 45823600}, {'date': '2025-11-21', 'close': np.float64(271.49), 'volume': 59030800}, {'date': '2025-11-24', 'close': np.float64(275.92), 'volume': 65585800}, {'date': '2025-11-25', 'close': np.float64(276.97), 'volume': 46914200}, {'date': '2025-11-26', 'close': np.float64(277.55), 'volume': 33431400}]}


## Step 1.2: Fundamentals Tool
Defining `get_asset_fundamentals` to retrieve key financial metrics (P/E, Market Cap, Sector) using `yfinance`. This helps the agent assess value and risk.


In [5]:
def get_asset_fundamentals(symbol: str, asset_type: str = "equity") -> Dict[str, Any]:
    """
    Fetch fundamental data (P/E, Market Cap, Sector, Summary) for an asset.
    
    Args:
        symbol (str): Ticker symbol (e.g., 'AAPL').
        asset_type (str): 'equity' or 'etf'. (Crypto has limited fundamentals in yf).
        
    Returns:
        dict: Key fundamentals or error message.
    """
    try:
        if asset_type.lower() == 'crypto' and not symbol.endswith('-USD'):
             symbol = f"{symbol}-USD"

        ticker = yf.Ticker(symbol)
        info = ticker.info
        
        # Extract only the most relevant fields to save context tokens
        fundamentals = {
            "symbol": symbol,
            "name": info.get("shortName", "Unknown"),
            "sector": info.get("sector", "Unknown"),
            "industry": info.get("industry", "Unknown"),
            "market_cap": info.get("marketCap", "N/A"),
            "trailing_pe": info.get("trailingPE", "N/A"),
            "forward_pe": info.get("forwardPE", "N/A"),
            "dividend_yield": info.get("dividendYield", "N/A"),
            "fifty_two_week_high": info.get("fiftyTwoWeekHigh", "N/A"),
            "fifty_two_week_low": info.get("fiftyTwoWeekLow", "N/A"),
            "summary": info.get("longBusinessSummary", "No summary available")[:300] + "..." # Truncate summary
        }
        
        return {
            "status": "success",
            "data": fundamentals
        }
        
    except Exception as e:
        return {"status": "error", "message": str(e)}


In [ ]:
# --- Test the tool immediately ---
print("Testing Tool 1.2 with MSFT...")
fund_result = get_asset_fundamentals("MSFT")
fund_result = json.dumps(fund_result, indent=4)
print(fund_result)

Testing Tool 1.2 with MSFT...
{'status': 'success', 'data': {'symbol': 'MSFT', 'name': 'Microsoft Corporation', 'sector': 'Technology', 'industry': 'Software - Infrastructure', 'market_cap': 3608802230272, 'trailing_pe': 34.481533, 'forward_pe': 32.47492, 'dividend_yield': 0.75, 'fifty_two_week_high': 555.45, 'fifty_two_week_low': 344.79, 'summary': "Microsoft Corporation develops and supports software, services, devices, and solutions worldwide. The company's Productivity and Business Processes segment offers Microsoft 365 Commercial, Enterprise Mobility + Security, Windows Commercial, Power BI, Exchange, SharePoint, Microsoft Teams, Security a..."}}


## Step 1.3: Technical Analysis Tool
Defining `calculate_technical_indicators` to compute Volatility (Standard Deviation) and Moving Averages. This gives the agent quantitative data to assess risk.


In [7]:
def calculate_technical_indicators(symbol: str, period: str = "3mo") -> Dict[str, Any]:
    """
    Calculate technical indicators (Volatility, SMA) for a symbol.
    
    Args:
        symbol (str): Ticker symbol.
        period (str): Data lookback period (default '3mo').
        
    Returns:
        dict: Calculated metrics including annualized volatility and current trend.
    """
    try:
        # We reuse our first tool logic to get the raw dataframe usually, 
        # but calling yfinance directly here is cleaner for the standalone tool.
        ticker = yf.Ticker(symbol)
        hist = ticker.history(period=period)
        
        if hist.empty:
             return {"status": "error", "message": "No data for calculations."}
        
        # 1. Calculate Daily Returns
        hist['Returns'] = hist['Close'].pct_change()
        
        # 2. Annualized Volatility (Standard Deviation of returns * sqrt(252 trading days))
        volatility = hist['Returns'].std() * np.sqrt(252)
        
        # 3. Simple Moving Average (50-day) - requiring enough data
        if len(hist) >= 50:
            sma_50 = hist['Close'].rolling(window=50).mean().iloc[-1]
        else:
            sma_50 = "Not enough data"
            
        current_price = hist['Close'].iloc[-1]
        
        # Determine trend
        trend = "Neutral"
        if isinstance(sma_50, (int, float)):
            if current_price > sma_50:
                trend = "Uptrend (Above 50d SMA)"
            else:
                trend = "Downtrend (Below 50d SMA)"
        
        return {
            "status": "success",
            "symbol": symbol,
            "current_price": round(current_price, 2),
            "annualized_volatility": round(volatility * 100, 2), # as percentage
            "sma_50": round(sma_50, 2) if isinstance(sma_50, float) else sma_50,
            "trend": trend
        }

    except Exception as e:
        return {"status": "error", "message": str(e)}


In [ ]:
# --- Test the tool ---
print("Testing Tool 1.3 with NVDA...")
tech_result = calculate_technical_indicators("NVDA")
tech_result = json.dumps(tech_result, indent=4)
print(tech_result)

Testing Tool 1.3 with NVDA...
{'status': 'success', 'symbol': 'NVDA', 'current_price': np.float64(180.26), 'annualized_volatility': np.float64(37.57), 'sma_50': np.float64(186.81), 'trend': 'Downtrend (Below 50d SMA)'}


## Step 2: Initialize Agent with Tools
We define the `tools` list containing our three functions and configure the Gemini Pro model to use them. We then start a chat session (Agent) that knows about these tools.


In [16]:
# 1. Define the Tool List
# We pass the actual function objects to the GenAI SDK. 
# The SDK automatically inspects the docstrings to understand how to use them.
my_tools = [
    get_asset_price_history,
    get_asset_fundamentals,
    calculate_technical_indicators
]

# 2. Initialize the Model with Tools
# We use 'gemini-1.5-flash' or 'gemini-1.5-pro' for best tool performance.
# 'auto' function calling mode is default, meaning the model decides when to use tools.
model = genai.GenerativeModel(
    model_name='gemini-2.5-flash', # Or 'gemini-1.5-pro' if you have access
    tools=my_tools
)

# 3. Create the Agent (Chat Session)
# enable_automatic_function_calling=True means the SDK handles the loop:
# Model asks for tool -> SDK runs tool -> SDK gives answer back to Model -> Model answers user.
agent = model.start_chat(enable_automatic_function_calling=True)



In [19]:
print("Agent initialized with Financial Tools!")

# --- Quick Test ---
# We ask a question that REQUIRES data to answer.
print("\nAsking Agent: 'What is the current volatility of Apple and is it in an uptrend?'")
response = agent.send_message("What is the current volatility of Apple (AAPL) and is it in an uptrend?")

print("\nAgent Answer:")
print(response.text)

Agent initialized with Financial Tools!

Asking Agent: 'What is the current volatility of Apple and is it in an uptrend?'

Agent Answer:
The current annualized volatility of Apple (AAPL) is 22.72%. Yes, it is in an uptrend as its current price of $277.55 is above its 50-day Simple Moving Average (SMA) of $261.63.


## Step 3: Defining the Financial Advisor Persona
We re-initialize the agent with a **System Instruction**. This tells the model to act as a professional Investment Advisor, mandates that it *must* use tools before answering, and defines risk rules (e.g., "High volatility > 30% requires a warning").


In [29]:
# Define the System Prompt (Persona & Rules)
advisor_instruction = advisor_instruction = """
You are an expert Financial Research Assistant and Investment Advisor.

YOUR GOAL:
Help the user analyze assets and construct portfolios based on data.

RULES:
1. ALWAYS fetch real data using your tools before answering. Do not guess.
2. If a user asks about a stock, get its Fundamentals AND Technicals.
3. RISK ASSESSMENT:
   - If Annualized Volatility is > 30%, label the asset as "High Risk".
   - If Volatility is < 15%, label it as "Stable/Low Risk".
4. RECOMMENDATIONS: Cite specific metrics to support your view.

FORMATTING REQUIREMENTS:
- You MUST present data in two distinct tables:
  
  **Table 1: Fundamentals**
  | Metric | Value |


  **Table 2: Technicals**
  | Metric | Value |

- Make sure the table is well format in a uniform spaced manner with its borders
- Do not leave tables empty. If data is missing, write "N/A".
"""

# Re-initialize model with the system instruction
advisor_model = genai.GenerativeModel(
    model_name='gemini-2.5-flash',
    tools=my_tools,
    system_instruction=advisor_instruction
)

# Start the Advisor Chat
advisor_agent = advisor_model.start_chat(enable_automatic_function_calling=True)

print("Advisor Agent initialized with System Instructions!")


Advisor Agent initialized with System Instructions!


In [30]:
# --- Test the Persona ---
query = "Analyze Tesla (TSLA). Is it safe for a conservative retiree?"
print(f"\nUser: {query}")
response = advisor_agent.send_message(query)

print("\nAdvisor Response:")
print(response.text)


User: Analyze Tesla (TSLA). Is it safe for a conservative retiree?

Advisor Response:
Here's an analysis of Tesla (TSLA):

**Table 1: Fundamentals**
| Metric            | Value               |
|:------------------|:--------------------|
| Company Name      | Tesla, Inc.         |
| Sector            | Consumer Cyclical   |
| Industry          | Auto Manufacturers  |
| Market Cap        | $1.42 T             |
| Trailing P/E      | 294.19              |
| Forward P/E       | 131.66              |
| Dividend Yield    | N/A                 |
| 52 Week High      | $488.54             |
| 52 Week Low       | $214.25             |

**Table 2: Technicals**
| Metric               | Value            |
|:---------------------|:-----------------|
| Current Price        | $426.58          |
| Annualized Volatility| 50.72%           |
| 50-day SMA           | $433.68          |
| Trend                | Downtrend (Below 50d SMA)|

**Risk Assessment:**
Based on an Annualized Volatility of 50.72%, Te

## Step 4: Portfolio Allocation Tool
Defining `allocate_portfolio` to simulate investment execution. This tool takes a list of assets and a budget, fetches real-time prices, and calculates an equal-weighted allocation (share counts and values).


In [31]:
def allocate_portfolio(tickers: list[str], total_amount: float) -> Dict[str, Any]:
    """
    Create an equal-weighted portfolio allocation for a list of tickers.
    
    Args:
        tickers (list[str]): List of symbols (e.g., ['AAPL', 'MSFT', 'BTC-USD']).
        total_amount (float): Total cash to invest (e.g., 10000.0).
        
    Returns:
        dict: Allocation plan including shares per asset and remaining cash.
    """
    try:
        if not tickers or total_amount <= 0:
            return {"status": "error", "message": "Invalid tickers or amount."}
        
        # 1. Determine target amount per asset (Equal Weight strategy)
        target_per_asset = total_amount / len(tickers)
        
        allocation = []
        total_spent = 0.0
        
        # 2. Fetch price and calculate shares for each asset
        for symbol in tickers:
            # Handle crypto formatting if user forgets
            search_symbol = symbol
            if symbol in ["BTC", "ETH"] or (symbol.isupper() and len(symbol) <= 4 and "USD" not in symbol and symbol not in ["AAPL", "MSFT", "TSLA", "NVDA", "GOOG", "AMZN"]): 
                 # Simple heuristic: if short ticker & not famous stock, try crypto suffix
                 # Ideally, the agent should pass correct symbols, but this makes it robust.
                 pass 
            
            # Use yfinance to get *current* price (efficiently)
            ticker_obj = yf.Ticker(search_symbol)
            # fast_info is faster than history() for current price
            try:
                price = ticker_obj.fast_info['last_price']
            except:
                # Fallback if fast_info fails
                hist = ticker_obj.history(period="1d")
                if not hist.empty:
                    price = hist['Close'].iloc[-1]
                else:
                    return {"status": "error", "message": f"Could not fetch price for {symbol}"}
            
            # Calculate shares (can be fractional for simplicity, or floor for whole shares)
            # We'll use 4 decimal places for precision
            shares = round(target_per_asset / price, 4)
            cost = round(shares * price, 2)
            
            allocation.append({
                "symbol": symbol,
                "price": round(price, 2),
                "shares": shares,
                "value": cost,
                "weight": f"{round((cost / total_amount) * 100, 1)}%"
            })
            total_spent += cost
            
        remaining_cash = round(total_amount - total_spent, 2)
        
        return {
            "status": "success",
            "strategy": "Equal Weight",
            "total_investment": total_amount,
            "allocation": allocation,
            "total_spent": round(total_spent, 2),
            "remaining_cash": remaining_cash
        }

    except Exception as e:
        return {"status": "error", "message": str(e)}



In [36]:
# --- Test the tool ---
print("Testing Portfolio Tool with $10k split between Apple, Nvidia, and Bitcoin...")
# Note: We pass a list of strings.
portfolio_result = allocate_portfolio(["AAPL", "NVDA", "BTC-USD"], 10000)
portfolio_result = json.dumps(portfolio_result, indent=4)
print(portfolio_result)

Testing Portfolio Tool with $10k split between Apple, Nvidia, and Bitcoin...
{
    "status": "success",
    "strategy": "Equal Weight",
    "total_investment": 10000,
    "allocation": [
        {
            "symbol": "AAPL",
            "price": 277.55,
            "shares": 12.0098,
            "value": 3333.32,
            "weight": "33.3%"
        },
        {
            "symbol": "NVDA",
            "price": 180.26,
            "shares": 18.4918,
            "value": 3333.33,
            "weight": "33.3%"
        },
        {
            "symbol": "BTC-USD",
            "price": 91341.24,
            "shares": 0.0365,
            "value": 3333.96,
            "weight": "33.3%"
        }
    ],
    "total_spent": 10000.61,
    "remaining_cash": -0.61
}


## Step 5: Final Agent Assembly
Updating the agent with the `allocate_portfolio` tool and refined instructions. The agent can now Research, Analyze Risk, AND Construct Portfolios in a single conversation.


In [41]:
# 1. Update Tool List
final_tools = [
    get_asset_price_history,
    get_asset_fundamentals,
    calculate_technical_indicators,
    allocate_portfolio  
]

# 2. Update System Instruction
final_instruction = """
You are an expert Financial Research Assistant and Investment Advisor.

YOUR CAPABILITIES:
1. **Research**: Fetch prices, fundamentals, and news (via RAG if available).
2. **Analysis**: Calculate technicals (volatility, trend) to assess risk.
3. **Portfolio Construction**: Create detailed allocation plans for users.

RULES:
- **Always use tools**. Never guess data.
- **Risk-First Approach**: If a user asks for a portfolio, FIRST check the risk (volatility) of the assets. If an asset is >30% volatility, warn the user before allocating.
- **Crypto Handling**: For crypto, always use the '-USD' suffix (e.g., BTC-USD, ETH-USD) when calling tools.
- **Tables**: Present Research data in the 'Fundamentals' and 'Technicals' tables defined previously with clear spaced formatting.
- **Portfolio Output**: When showing a portfolio, use a table with columns: Asset | Price | Shares | Value | Weight.

DISCLAIMER: Always end portfolio recommendations with: *"This is a simulation for educational purposes."*
"""

# 3. Initialize Final Agent
final_model = genai.GenerativeModel(
    model_name='gemini-2.5-flash',
    tools=final_tools,
    system_instruction=final_instruction
)

# Start the Chat
analyst_desk = final_model.start_chat(enable_automatic_function_calling=True)

print("🚀 Final Analyst Desk is Ready!")




🚀 Final Analyst Desk is Ready!


In [42]:
# --- FINAL BOSS BATTLE (Complex Query) ---
query = """
I have $50,000 to invest. 
I want a portfolio with Microsoft (MSFT), Nvidia (NVDA), and Bitcoin (BTC).
Please analyze the risk of these assets first, and then generate an allocation plan for me.
"""

print(f"\n👤 User: {query}")
response = analyst_desk.send_message(query)

print("\n🤖 Analyst Response:")
print(response.text)


👤 User: 
I have $50,000 to invest. 
I want a portfolio with Microsoft (MSFT), Nvidia (NVDA), and Bitcoin (BTC).
Please analyze the risk of these assets first, and then generate an allocation plan for me.


🤖 Analyst Response:
Here is your equally-weighted portfolio allocation plan:

**Portfolio Allocation**

| Asset   | Price     | Shares   | Value       | Weight |
| :------ | :-------- | :------- | :---------- | :----- |
| MSFT    | $485.50   | 34.33    | $16,666.68  | 33.3%  |
| NVDA    | $180.26   | 92.46    | $16,666.66  | 33.3%  |
| BTC-USD | $91,479.53 | 0.18     | $16,667.57  | 33.3%  |

Total Investment: $50,000.00
Remaining Cash: $-0.91

This is a simulation for educational purposes.
